In [11]:
# 모듈 임포트
import time
import board
import busio
import adafruit_ads1x15.ads1115 as ADS
from adafruit_ads1x15.analog_in import AnalogIn
import joblib
import numpy as np
import psutil
import pandas as pd
import csv
from datetime import datetime
import tflite_runtime.interpreter as tflite
from sklearn.preprocessing import StandardScaler

In [12]:
# TFLite 모델 및 스케일러 로드
interpreter = tflite.Interpreter(model_path="./model/cnn1d_model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

scaler = joblib.load('./model/scaler_cnn.joblib')

In [13]:
# I2C 및 ADC 설정
i2c = busio.I2C(board.SCL, board.SDA)
ads = ADS.ADS1115(i2c)
ads.gain = 2/3

voltage_ch = AnalogIn(ads, ADS.P0)
current_ch = AnalogIn(ads, ADS.P1)

In [14]:
# 센서 보정값
ZMPT_offset = 2.5539
ZMPT_scale = 997.6

ACS712_offset = 2.5087
ACS712_sensitivity = 0.0990

In [15]:
def extract_features_from_sequence(seq):
    voltage_seq, current_seq = zip(*seq)
    voltage = np.mean(voltage_seq)
    current = np.mean(current_seq)
    voltage_diff = np.abs(voltage_seq[-1] - voltage_seq[-2])
    current_diff = np.abs(current_seq[-1] - current_seq[-2])
    voltage_ma = np.mean(voltage_seq[-5:])
    current_ma = np.mean(current_seq[-5:])
    power = voltage * current
    power_diff = np.abs(power - (voltage_seq[-2] * current_seq[-2]))
    return [voltage, current, voltage_diff, current_diff, voltage_ma, current_ma, power, power_diff]

In [ ]:
def build_cnn_input(data_buffer, seq_len=8):
    sequence = []
    if len(data_buffer) < seq_len + 7:
        return None

    # i = 0부터 7까지 8개의 시퀀스를 만들도록 조정
    for i in range(8):
        window = data_buffer[i:i+8]
        if len(window) < 8:
            continue
        voltage_seq, current_seq = zip(*window)
        features = extract_features_from_sequence(list(zip(voltage_seq, current_seq)))
        sequence.append(features)

    return np.array(sequence) if len(sequence) == 8 else None


In [17]:
def postprocess_anomalies_realtime(anomaly_buffer, min_consecutive=8):
    if len(anomaly_buffer) < min_consecutive:
        return False
    count = 0
    for val in reversed(anomaly_buffer):
        if val == 1:
            count += 1
        else:
            break
    return count >= min_consecutive

In [18]:
# 테스트 데이터 불러오기 (센서 대체)
column_names = ['timestamp', 'v_raw', 'c_raw', 'voltage', 'current']
df = pd.read_csv('./log/raw_dataset_arc.csv', names=column_names, header=None)
df = df.dropna(subset=['voltage', 'current']).reset_index(drop=True)
csv_filename = "./log/cnn_log1.csv"

In [ ]:
try:
    print("🚀 실시간 아크 감지 시작합니다!")

    USE_SENSOR = True
    ANOMALY_THRESHOLD = 2

    data_buffer = []
    anomaly_buffer = []
    first_anomaly_detected = False
    anomaly_start_time = None
    start_time = time.perf_counter()
    idx = -1

    while True:
        idx += 1

        if USE_SENSOR:
            try:
                v_raw_sensor = voltage_ch.voltage
                c_raw_sensor = current_ch.voltage
                voltage = (v_raw_sensor - ZMPT_offset) * ZMPT_scale
                current = (c_raw_sensor - ACS712_offset) / ACS712_sensitivity

                prev_voltage = voltage
                prev_current = current

            except (OSError, ValueError, RuntimeError) as e:
                print(f"⚠️ 센서 오류 발생: {e}")
                voltage = prev_voltage if 'prev_voltage' in locals() else 0
                current = prev_current if 'prev_current' in locals() else 0

        else:
            voltage = df['voltage'].iloc[idx]
            current = df['current'].iloc[idx]

        data_buffer.append((voltage, current))
        if len(data_buffer) > 100:
            data_buffer.pop(0)

        # CSV 기록
        with open(csv_filename, mode='a', newline='') as file:
            writer = csv.writer(file)
            timestamp = datetime.now().isoformat()
            writer.writerow([timestamp, voltage, current])

        if len(data_buffer) >= 15:
            X_seq = build_cnn_input(data_buffer)
            if X_seq is None:
                continue  # 데이터가 부족할 때는 건너뜀
            # X_seq = build_cnn_input(data_buffer)
            X_scaled = scaler.transform(X_seq)
            X_scaled = X_scaled.reshape(1, 8, 8)  # CNN 입력
            

            # 입력값 설정
            interpreter.set_tensor(input_details[0]['index'], X_scaled.astype(np.float32))
            interpreter.invoke()
            output_data = interpreter.get_tensor(output_details[0]['index'])
            y_pred = output_data[0][0]
            anomaly = int(y_pred > 0.5)
            # y_pred = model.predict(X_scaled, verbose=0)[0][0]
            # anomaly = int(y_pred > 0.5)

            anomaly_buffer.append(anomaly)
            if len(anomaly_buffer) > ANOMALY_THRESHOLD:
                anomaly_buffer.pop(0)

            if anomaly == 1 and anomaly_start_time is None:
                anomaly_start_time = time.perf_counter()
            elif anomaly == 0:
                anomaly_start_time = None

            if postprocess_anomalies_realtime(anomaly_buffer, min_consecutive=ANOMALY_THRESHOLD):
                print("⚡ 아크 이상 감지!", idx)
                if not first_anomaly_detected and anomaly_start_time is not None:
                    total_elapsed = time.perf_counter() - start_time
                    actual_elapsed = time.perf_counter() - anomaly_start_time
                    print(f"⏱️ 전체 소요 시간: {total_elapsed:.6f}초")
                    print(f"⏱️ 연속 8개 이상 신호까지 실제 소요 시간: {actual_elapsed:.6f}초")
                    first_anomaly_detected = True
                    break

        cpu_usage = psutil.cpu_percent(interval=0.0)
        memory_usage = psutil.Process().memory_info().rss / 1024 ** 2
        print(f"🧠 메모리: {memory_usage:.2f}MB | 🧮 CPU: {cpu_usage:.2f}% | idx: {idx}")
        print(f"v_raw:{v_raw_sensor}, c_raw:{c_raw_sensor}, voltage:{voltage}, current:{current}")

        time.sleep(0.001163)

except KeyboardInterrupt:
    print("🛑 실시간 감지를 종료합니다.")

🚀 실시간 아크 감지 시작합니다!
🧠 메모리: 178.95MB | 🧮 CPU: 4.40% | idx: 0
v_raw:2.4939375, c_raw:2.5404375, voltage:-59.81859000000014, current:0.3205808080808056
🧠 메모리: 178.95MB | 🧮 CPU: 45.50% | idx: 1
v_raw:2.5801875, c_raw:2.5408125, voltage:26.22441000000002, current:0.3243686868686845
🧠 메모리: 178.95MB | 🧮 CPU: 63.60% | idx: 2
v_raw:2.5644375, c_raw:2.540625, voltage:10.512209999999866, current:0.322474747474745
🧠 메모리: 178.95MB | 🧮 CPU: 10.00% | idx: 3
v_raw:2.5831875, c_raw:2.5404375, voltage:29.217210000000133, current:0.3205808080808056
🧠 메모리: 178.95MB | 🧮 CPU: 10.00% | idx: 4
v_raw:2.568, c_raw:2.540625, voltage:14.066160000000002, current:0.322474747474745
🧠 메모리: 179.07MB | 🧮 CPU: 44.40% | idx: 5
v_raw:2.5471875, c_raw:2.5408125, voltage:-6.6963899999999, current:0.3243686868686845
🧠 메모리: 179.07MB | 🧮 CPU: 11.10% | idx: 6
v_raw:2.562375, c_raw:2.5411875, voltage:8.45465999999979, current:0.3281565656565634
🧠 메모리: 179.07MB | 🧮 CPU: 0.00% | idx: 7
v_raw:2.56425, c_raw:2.5404375, voltage:10.325

ValueError: cannot reshape array of size 56 into shape (1,8,8)